# Pipeline to explore the web analytics data

In [1]:
import os
import re
from io import StringIO

import pandas as pd

This script is fed by downloading from Google Analytics by navigating to **Reports -> Engagement -> Events**, then selecting page_view. Make sure the date range is as required (starting on Saturday 5th November 2022, and running to the current day) Click __Share this report__. Download a CSV and then upload to [files.open-innovations.org](https://files.open-innovations.org) and place in the `web_analyics` folder in the `leeds2023` user account.

The following loop processes transforms this into a series of pandas data frames.

* daily_data - data about overall number of visits per day (sometimes not available)
* weekly_data - data about overall number of visits per week
* page_data - information about the pages viewed on the site

In [2]:
WEEKLY_DATA='weekly_data'
DAILY_DATA='daily_data'
PAGE_DATA='page_data'

with open('../../../working/manual/web_analytics/data-export.csv') as f:
    mode = None
    weekly_data = pd.DataFrame()
    daily_data = pd.DataFrame()
    page_data = pd.DataFrame()
    for line in f.readlines():
        if line.startswith('# Start date'):
            start_date = pd.to_datetime(re.search(r'[0-9]+', line).group(0))
            continue
        if line.startswith('Nth week'):
            mode = WEEKLY_DATA
            csv = ''
        if line.startswith('Nth day'):
            mode = DAILY_DATA
            csv = ''
        if line.startswith('Page title and screen name'):
            mode = PAGE_DATA
            csv = ''
        if re.match(r'^$', line):
            if mode == WEEKLY_DATA:
                new_data = pd.read_csv(StringIO(csv), index_col=0)
                new_data.index = pd.DatetimeIndex(start_date + new_data.index.map(lambda x: pd.DateOffset(weeks=x)))
                new_data.index.name = 'week'
                new_data.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)
                weekly_data = pd.concat([weekly_data, new_data], axis=1)
            if mode == DAILY_DATA:
                new_data = pd.read_csv(StringIO(csv), index_col=0)
                new_data.index = pd.DatetimeIndex(start_date + new_data.index.map(lambda x: pd.DateOffset(days=x)))
                new_data.index.name = 'day'
                new_data.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)
                daily_data = pd.concat([daily_data, new_data], axis=1)
            if mode == PAGE_DATA:
                new_data = pd.read_csv(StringIO(csv))
                new_data.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)
                page_data = pd.concat([page_data, new_data])
            mode = None
        if mode != None:
            csv += line

Clean up the data

In [3]:
if (len(daily_data.index) > 0):
    daily_data = daily_data[['event_count', 'total_users']]
    weekly_data = daily_data.resample('W-FRI').sum()


Set up the output directory

In [4]:
DATA_DIR = os.path.join('../../../data/metrics', 'web_analytics')
os.makedirs(DATA_DIR, exist_ok=True)

Save the CSV files

In [5]:
daily_data.to_csv(os.path.join(DATA_DIR, 'daily_page_views.csv'))
weekly_data.to_csv(os.path.join(DATA_DIR, 'weekly_page_views.csv'))